In [1]:
import sys
sys.path.append("../python")

import json
import h5py
import numpy as np
import arviz as az
import compute_costs
import evaluations

KeyboardInterrupt: 

In [ ]:
settings = [
    "MH",
    "MLDA ref",
    "MLDA test"
]

parameters_paths = [
    "../configs/test_mh/mh.json",
    "../configs/test_mlda/mlda_ref.json",
    "../configs/test_mlda/mlda_test.json"
]

samples = []
chains = []
num_samples = []
burn_in = []
costs = []
    
for path in parameters_paths:
    file = open(path)
    config = json.load(file)
    file.close()

    num_samples.append(config["Sampling"]["NumSamples"])
    burn_in.append(config["Sampling"]["BurnIn"])
    chains.append(config["Setup"]["Chains"])
    
    samples_mlda = []
    for c in range(config["Setup"]["Chains"]):
        sample_path = config["Setup"]["OutputPath"] + config["Sampling"]["ResultFile"] 
        fin = h5py.File(sample_path + "_" + str(c) + ".h5")
        samples_mlda.append(np.array(fin['/samples']))
        fin.close()
    samples.append(samples_mlda)

In [ ]:
# compute ess
ess = []

for i,sample in enumerate(samples):
    ess_mlda = []
    for j,chain in enumerate(sample): 
        if(evaluations.is_converged(chain)==False): 
            print("Not converged in setting " + settings[i] + " for chain " + str(j))  
        elif(evaluations.burn_in_check(chain)==False): 
            print("Burn-in too small in setting " + settings[i] + " for chain " + str(j))  
        ess_mlda.append(min(az.ess(chain[0]),az.ess(chain[1]),az.ess(chain[2])))
    ess.append(ess_mlda)



In [ ]:
print("Samples (on finest level) per effective sample \n")

samples_per_es = np.zeros((len(settings),min(chains)))
rel_samples_per_es = np.zeros((len(settings),min(chains)))

for j in range(min(chains)):
    for i in range(len(settings)):
        samples_per_es[i,j] = (num_samples[i]-burn_in[i])/ess[i][j]
        rel_samples_per_es[i,j] = samples_per_es[i,j]/samples_per_es[0,j]

        print(settings[i] + ": " + str(samples_per_es[i,j]) + " (" + str(rel_samples_per_es[i,j]) +")")
            
    print("\n")



Samples (on finest level) per effective sample 

MH: 229.09492849636047 (1.0)
MLDA ref: 66.66914515484044 (0.29101100400788477)


MH: 240.52866358689565 (1.0)
MLDA ref: 105.02775269916353 (0.4366537905833423)


MH: 496.3669327102241 (1.0)
MLDA ref: 236.96464776761604 (0.4773981346294769)


MH: 5959.780849576767 (1.0)
MLDA ref: 167.5273906112812 (0.028109656183612546)


MH: 1679.498619298118 (1.0)
MLDA ref: 200.47776749603793 (0.11936762864373174)


MH: 1905.144002709897 (1.0)
MLDA ref: 3400.849670270056 (1.7850879857022102)


MH: 16982.813421078394 (1.0)
MLDA ref: 288.97689430197806 (0.017015843437537353)




In [ ]:
costs = np.zeros(len(parameters_paths))
for i in range(len(settings)):
    costs[i] = evaluations.costs_per_sample(parameters_paths[i])

Dipole:
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
volume_conductor.time_creating_grid = 0.20263
volume_conductor.time_reordering_labels = 0.0026615
volume_conductor.time = 0.205293
time_element_neighborhood_map 0.0139845 s
degree = 1
element_type = hexahedron
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
sigma = 0.11508410052537331
{'Level3_V': {0: array([-8.55226161, -8.56861298, -8.14924543, -8.60904547, -8.91536296,
       -8.28212764, -8.06684611, -9.23144628, -7.87700115, -9.24694914,
       -7.92711236, -9.23213438, -9.13928853, -7.79511614, -9.15710694,
       -8.44429476, -8.16618367, -7.40134013, -6.20391782, -3.90283444,
       -0.16406144,  6.07622764, 13.82840884, 20.44844333, 23.09616768,
       21.93330918, 19.34472515, 16.45954569, 13.75498795, 11.79171568,
       10.13486827,  2.82477786,  2.24388962,  1.51179187, -0.22799063,
        0.07822417])}}
volume_conductor.time_creating_grid = 0.174751
volume_conductor.time_reordering_labels = 0.00299

In [ ]:
print("Costs per effective sample \n")

costs_per_ed = np.zeros((len(settings),min(chains)))
rel_costs_per_ed = np.zeros((len(settings),min(chains)))

for j in range(min(chains)):
    for i in range(len(settings)):
        costs_per_ed[i,j] = costs[i]/ess[i][j]
        rel_costs_per_ed[i,j] = costs_per_ed[i,j] / costs_per_ed[0,j]

        print(settings[i] + ": " + str(costs_per_ed[i,j]) + " (" + str(rel_costs_per_ed[i,j]) + ")")
            
    print("\n")

Costs per effective sample 

MH: 0.8192480467433901 (1.0)
MLDA ref: 1.2600446165141679 (1.5380501931289277)


MH: 0.8601353121288878 (1.0)
MLDA ref: 1.985021017830675 (2.3078008655610547)


MH: 1.775018079884379 (1.0)
MLDA ref: 4.478623927609819 (2.523142709566962)


MH: 21.31229552779906 (1.0)
MLDA ref: 3.1662620867291102 (0.14856504230617212)


MH: 6.005920656554658 (1.0)
MLDA ref: 3.789023109227929 (0.6308813129412087)


MH: 6.8128330610765255 (1.0)
MLDA ref: 64.27594517141702 (9.434539874262015)


MH: 60.73088049021102 (1.0)
MLDA ref: 5.461653649773165 (0.08993206760197571)




In [ ]:
print("Average relative costs per effective sample \n")

for i in range(len(settings)):
    av = 0
    for j in range(min(chains)):  
        av += rel_costs_per_ed[i,j]
    av = av/min(chains)
    print(settings[i] + ": " + str(av))


Average relative costs per effective sample 

MH: 1.0
MLDA ref: 2.381844580766902
